<h1 style="text-align:center;"><strong>Affiliation Search vs Article Search<strong></h1>

<h3><strong>Parte 1: Importar las bibliotecas y configurar el cliente</strong></h3>

<h4 style="text-align:justify;">Paso 1: Importar librerías</h4>

In [2]:
import json
import time
import mysql.connector
import pandas as pd
import numpy as np
from ScopusModule.Client import Client
from ScopusModule.Search import Search
from ScopusModule.Retrieval import AuthorRetrieval, ArticleRetrieval, AffilRetrieval
from ScopusModule import Utils
from sqlalchemy import create_engine
from urllib.parse import quote_plus as url_encode

SyntaxError: invalid syntax (Temp/ipykernel_10060/3043838915.py, line 6)

<h4 style="text-align:justify;">Paso 2: Cargar el archivo de configuración</h4>

In [11]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

<h4 style="text-align:justify;">Paso 3: Configurar el cliente para realizar peticiones</h4>

In [12]:
client = Client(api_key = config['apikey'], inst_token = config['insttoken'])

In [13]:
#Configuración del cliente
print('apikey:', client.api_key)
print('insttoken:', client.inst_token)

apikey: 27b4573814803b246abfac3615c505e0
insttoken: 330aca6833fdfff081a7bd4d1ed7392f


<h4 style="text-align:justify;">Paso 4: Definir la ruta del directorio</h4>

In [14]:
directorio = "27-06-2022"

<hr>

<hr>

<h3><strong>Parte 2: Cargar la Data</strong></h3>

<h4 style="text-align:justify;">Paso 1: Leer el pickle del dataframe de artículos</h4>

In [15]:
dfArticles = pd.read_pickle('data/completo/'+directorio+'/article.pkl')

In [16]:
dfArticles.head(3)

,identifier,title,publication_date,prism:doi,abstract,affiliations,authors,author_keywords,author_count,affiliation_count
0,85112575431,"Corporate social responsibility, green innovat...",2022-12-19,10.1108/CR-12-2020-0160,Purpose: The purpose of this paper is to explo...,"[{'afid': '60072050', 'affiliation-country': '...","[{'authid': '57195298226', 'afid': ['60072050'...",Competitiveness | Corporate social responsibil...,2,2
1,85109263966,Analysing competing logics towards sustainable...,2022-12-19,10.1108/SCM-07-2020-0354,Purpose: To understand the context surrounding...,"[{'afid': '60104441', 'affiliation-country': '...","[{'authid': '57190878471', 'afid': ['60023256'...",Food industry | Global supply chain | Supply c...,3,3
2,85123598971,Lumbocostovertebral syndrome. A case report,2022-12-17,10.30944/20117582.907,Introduction. Lumbocostovertebral syndrome is ...,"[{'afid': '60072035', 'affiliation-country': '...","[{'authid': '57223889019', 'afid': ['60072035'...",Abdominal hernia | Congenital abnormalities | ...,4,2


<h4 style="text-align:justify;">Paso 2: Obtener los IDs de las afiliaciones ecuatorianas extraidas de los artículos</h4>

In [17]:
affiliationsIdsRetrieval = []
for index, row in dfArticles.iterrows():
    for affil in row['affiliations']:
        if(affil['affiliation-country'] == "Ecuador"):
            affiliationsIdsRetrieval.append(affil['afid'])

In [18]:
affiliationsIdsRetrieval = list(dict.fromkeys(affiliationsIdsRetrieval))

In [19]:
len(affiliationsIdsRetrieval)

5218

<h4 style="text-align:justify;">Paso 3: Leer el pickle del dataframe de las afiliaciones</h4>

In [20]:
dfAffiliations = pd.read_pickle('data/completo/'+directorio+'/full_affiliation_search_sin_duplicados.pkl')
dfAffiliations.head(3)

,@_fa,prism:url,dc:identifier,affiliation-name,document-count,city,country
0,true,https://api.elsevier.com/content/affiliation/a...,AFFILIATION_ID:60072059,Universidad San Francisco de Quito,3731,Quito,Ecuador
1,true,https://api.elsevier.com/content/affiliation/a...,AFFILIATION_ID:60072054,Escuela Politécnica Nacional,3194,Quito,Ecuador
2,true,https://api.elsevier.com/content/affiliation/a...,AFFILIATION_ID:60072061,Escuela Superior Politecnica del Litoral Ecuador,2802,Guayaquil,Ecuador


<h4 style="text-align:justify;">Paso 4: Obtener los IDs de las afiliaciones que no fueron extraídas</h4>

In [21]:
affiliationsIdsSearch = dfAffiliations.loc[:,'dc:identifier'].tolist()

In [22]:
affiliationsIdsSearch = [affilId.split(':')[1] for affilId in affiliationsIdsSearch]

In [23]:
len(affiliationsIdsSearch)

5000

In [24]:
affiliationsIdsDiff = np.setdiff1d(affiliationsIdsRetrieval,affiliationsIdsSearch)

In [25]:
len(np.setdiff1d( affiliationsIdsSearch, affiliationsIdsRetrieval))

29

In [26]:
len(affiliationsIdsDiff)

247

In [27]:
len(np.intersect1d(affiliationsIdsSearch, affiliationsIdsRetrieval))

4971

<h4 style="text-align:justify;">Paso 5: Leer las afiliaciones ecuatorianas extraídas a través de la API affiliation retrieval</h4>


In [28]:
dfAffiliationsRetrieval = pd.read_pickle('data/completo/'+directorio+'/full_affiliation_retrieval.pkl')

In [29]:
dfAffiliationsRetrieval.head(3)

,coredata,affiliation-name,country,city
0,"{'dc:identifier': 'AFFILIATION_ID:100391348', ...",Facultad de Ciencias Médicas,Ecuador,NaN
1,"{'dc:identifier': 'AFFILIATION_ID:100410297', ...","Centro de IFMC, Corp. de Fis. Fundamental y Ap...",Ecuador,Quito
2,"{'dc:identifier': 'AFFILIATION_ID:100463995', ...",Facultad Latino Americana de Ciencias Sociales,Ecuador,Quito


In [30]:
len(dfAffiliationsRetrieval)

247

<h4 style="text-align:justify;">Paso 6: Recast de las afiliaciones obtenias a través de la API Affiliation Retrieval</h4>

In [31]:
dfAffiliationsRetrieval['identifier'] = dfAffiliationsRetrieval['coredata'].apply(lambda x: x["dc:identifier"].split(':')[1] 
                                                          if "dc:identifier" in x else np.nan)

In [32]:
dfAffiliationsRetrieval['document-count'] = dfAffiliationsRetrieval['coredata'].apply(lambda x: x["document-count"] 
                                                          if "document-count" in x else np.nan)

In [33]:
dfAffiliationsRetrieval.pop("coredata")

0      {'dc:identifier': 'AFFILIATION_ID:100391348', ...
1      {'dc:identifier': 'AFFILIATION_ID:100410297', ...
2      {'dc:identifier': 'AFFILIATION_ID:100463995', ...
3      {'dc:identifier': 'AFFILIATION_ID:100713966', ...
4      {'dc:identifier': 'AFFILIATION_ID:100762656', ...
                             ...                        
242    {'dc:identifier': 'AFFILIATION_ID:101865030', ...
243    {'dc:identifier': 'AFFILIATION_ID:101865309', ...
244    {'dc:identifier': 'AFFILIATION_ID:101869248', ...
245    {'dc:identifier': 'AFFILIATION_ID:101871025', ...
246    {'dc:identifier': 'AFFILIATION_ID:101874549', ...
Name: coredata, Length: 247, dtype: object

In [34]:
dfAffiliationsRetrieval.head(3)

,affiliation-name,country,city,identifier,document-count
0,Facultad de Ciencias Médicas,Ecuador,NaN,100391348,11
1,"Centro de IFMC, Corp. de Fis. Fundamental y Ap...",Ecuador,Quito,100410297,2
2,Facultad Latino Americana de Ciencias Sociales,Ecuador,Quito,100463995,1


<hr>

<hr>

<h3><strong>Parte 3: Verificar que afiliciones no fueron extraídas con la API Affiliation Search<strong></h3>

<h4 style="text-align:justify;">Paso 1: Mostrar las afiliaciones extraías</h4>

In [35]:
len(affiliationsIdsDiff)

247

In [36]:
for index, row in dfAffiliationsRetrieval.iterrows():
    #if int(row['document-count']) > 1:
    print('identifier:', row['identifier'])
    print('affiliation-name:', row['affiliation-name'])
    print('city:', row['city'])
    print('country:', row['country'])
    print('document-count:', row['document-count'] ,'\n')

identifier: 100391348
affiliation-name: Facultad de Ciencias Médicas
city: nan
country: Ecuador
document-count: 11 

identifier: 100410297
affiliation-name: Centro de IFMC, Corp. de Fis. Fundamental y Aplicada
city: Quito
country: Ecuador
document-count: 2 

identifier: 100463995
affiliation-name: Facultad Latino Americana de Ciencias Sociales
city: Quito
country: Ecuador
document-count: 1 

identifier: 100713966
affiliation-name: Social Security Hospital
city: Quito
country: Ecuador
document-count: 0 

identifier: 100762656
affiliation-name: Inst. Nac. Hig.
city: Guayaquil
country: Ecuador
document-count: 0 

identifier: 100778967
affiliation-name: CAAP
city: Quito
country: Ecuador
document-count: 1 

identifier: 100798427
affiliation-name: Oficina 702
city: Quito
country: Ecuador
document-count: 0 

identifier: 100803609
affiliation-name: PGU-ALC Programa de Gestion Urbana
city: Quito
country: Ecuador
document-count: 0 

identifier: 100812700
affiliation-name: IFA Corp. Devmt. Prod./

identifier: 101733901
affiliation-name: Hosp. Provincial del Seguro Social
city: nan
country: Ecuador
document-count: 1 

identifier: 101734140
affiliation-name: Fundación Bromelia
city: Quito
country: Ecuador
document-count: 1 

identifier: 101736467
affiliation-name: Comision Fulbright del Ecuador
city: Quito
country: Ecuador
document-count: 0 

identifier: 101740413
affiliation-name: Centro De Investigación En Física De La Materia Condensada, Corporación De Física
city: nan
country: Ecuador
document-count: 0 

identifier: 101749421
affiliation-name: Programa Bilingüe
city: nan
country: Ecuador
document-count: 1 

identifier: 101752035
affiliation-name: Instituto de Epidemiología y Salud Comunitaria 'Manuel Amuná rriź
city: nan
country: Ecuador
document-count: 0 

identifier: 101759057
affiliation-name: ICD
city: Guayaquil
country: Ecuador
document-count: 1 

identifier: 101759313
affiliation-name: Corp. Desarrollo e Invest. Geologico
city: Quito
country: Ecuador
document-count: 0 



<h4 style="text-align:justify;">Paso 2: Razones</h4>

<table>
  <tr>
    <th>ID</th>
    <th>Razón</th>
    <th>Afiliación</th>
    <th>Afiliacion en artículos</th>
  </tr>
  <tr>
    <td>100391348</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. El nombre de la afiliacion no concuerda. El document-count no concuerda</td>
    <td>Facultad de Ciencias Médicas, Rosario, U.N.R, Ecuador</td>
    <td>Facultad de Ciencias Medicas, Federico Proano, Cuenca, Azuay, Ecuador</td>
  </tr>
  <tr>
    <td>100410297</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. El document-count no concuerda</td>
    <td>Centro de IFMC, Corp. de Fis. Fundamental y Aplicada, Apartado 17-12-637, Quito, Ecuador</td>
    <td>CIFMC, Corp. de Fisica Fundamental y Apl., Quito, Apartado 17-12-637, Ecuador</td>
  </tr>
  <tr>
    <td>100463995</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search.</td>
    <td>Facultad Latino Americana de Ciencias Sociales, Quito, Ecuador</td>
    <td>Facultad Latino Americana de Ciencias Sociales, Quito, Ecuador</td>
  </tr>
   <tr>
    <td>100713966</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. El document-count no concuerda</td>
    <td>Social Security Hospital, Av. America y Asuncion, Quito, Ecuador</td>
    <td>Oncology and Endocrinology Services, Social Security Hospital, Quito, Av. America y Asuncion, Ecuador</td>
  </tr>
  <tr>
    <td>100762656</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. La afiliacion no concuerda. El document-count no concuerda. La afiliacion en los artículos no está relacionado con ningun autor.</td>
    <td>Inst. Nac. Hig., Guayaquil, Ecuador</td>
    <td>Ecuador</td>
  </tr>
  <tr>
    <td>100778967</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. La afiliacion no concuerda. La afiliacion en los artículos no está relacionado con ningun autor.</td>
    <td>CAAP, Apartado 173-B, Quito, Ecuador</td>
    <td>Ecuador</td>
  </tr>
  <tr>
    <td>100798427</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. El document-count no concuerda.</td>
    <td>Oficina 702, Quito, Ecuador</td>
    <td>BIOX, Edificio Citiplaza, Quito, Oficina 702, Ecuador</td>
  </tr>
  <tr>
    <td>100812700</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. El document-count no concuerda.</td>
    <td>IFA Corp. Devmt. Prod./Wk. Environ., Urbanizacion Granda Centeno, Domingo Brieva N38-107/Villalengua, Quito, Ecuador
</td>
    <td>IFA Corp. Devmt. Prod./Wk. Environ., Urbanización Granda Centeno, Quito, Domingo Brieva N38-107/Villalengua, Ecuador</td>
  </tr>
  <tr>
    <td>101104591</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. La afiliacion no concuerda. El document-count no concuerda.</td>
    <td>University San Francisco de Quito, Puerto Baquerizo Moreno, Galapagos, San Francisco, Ecuador</td>
    <td>Department of Dermatology, Hospital San Francisco de Quito, Ecuador</td>
  </tr>
  <tr>
    <td>101470614</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. El document-count no concuerda.</td>
    <td>Escla. Univ. de Enfermería, Area de Salud Publica, Edificio Melchor Cano, Camino de Pozuelo, s/n, Cuenca, Ecuador</td>
    <td>Escla. Univ. de Enfermería, Área de Salud Pública, Edificio Melchor Cano, 16002 Cuenca, Camino de Pozuelo, s/n, Ecuador</td>
  </tr>
  <tr>
    <td>101853811</td>
    <td>Afiliacion con un solo documento. Posiblemente corresponde a las afiliaciones no extraídas por la quota de la API Affiliation Search. El document-count no concuerda.</td>
    <td>Hospital General de las FF. AA. Quito, Ecuador</td>
    <td>Hospital General de las FF. AA. Quito, Ecuador</td>
  </tr>
</table>

<h4 style="text-align:justify;">Paso 3: Verificar cuantos artículos se obtienen</h4>

In [37]:
inicioArticle = time.time()
dataArticlesSearch = []
#Tipo de búsqueda
searchType = "scopus"
#Lista de elementos que se devolverán en la respuesta
view = "COMPLETE"
#Nombre de campos específicos que deben retornarse
field = "dc:identifier,doi,dc:title,coverDate,dc:description,authkeywords,afid,affiliation-country,authid"
#Número máximo de resultados que se devolverán por petición
count = "25"
#Parámetro se utiliza para ejecutar una búsqueda de paginación profunda
cursor = '*'
for group in Utils.chunker(affiliationsIdsDiff, 1):
    #Búsqueda booleana que se ejecutará en el clúster de SCOPUS.
    query = url_encode(" OR ".join(["AF-ID("+item+")" for item in group]))
    #Url de búsqueda de artículos
    url = "https://api.elsevier.com/content/search/"+searchType+"?query="+query+"&count="+count+"&view="+view+"&field="+field+"&cursor="+cursor
    #Declarar el objeto para la búsqueda de artículos
    articleSearch = Search(url=url)
    #Ejecución de la(s) petición(es)
    articleSearch.execute(client, True)
    #Concatenación de los resultados
    dataArticlesSearch = dataArticlesSearch + articleSearch.results
    print("Resultados totales actuales: ", len(dataArticlesSearch))
finArticle = time.time()

Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  1
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  2
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  3
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  4
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  5
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  6
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  7
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  8
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  9
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  10
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  11
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  12
Resultados totales: 1
Resultados actuales:  1
Resultados tota

Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  105
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  106
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  107
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  108
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  109
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  110
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  111
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  112
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  113
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  114
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  115
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  116
Resultados totales: 1
Resultados actuale

Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  208
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  209
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  210
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  211
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  212
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  213
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  214
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  215
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  216
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  217
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  218
Resultados totales: 1
Resultados actuales:  1
Resultados totales actuales:  219
Resultados totales: 1
Resultados actuale

In [38]:
print('Fecha de inicio:', time.ctime(inicioArticle)) 
print('Fecha de fin:', time.ctime(finArticle)) 
print('Tiempo total de ejecución:', finArticle-inicioArticle, 'segundos.') 

Fecha de inicio: Wed May 11 20:36:15 2022
Fecha de fin: Wed May 11 20:43:21 2022
Tiempo total de ejecución: 426.22082686424255 segundos.


In [39]:
#Resultados en un dataframe
dfArticlesRetrieval = pd.DataFrame(dataArticlesSearch)
dfArticlesRetrieval

,@_fa,prism:url,dc:identifier,dc:title,prism:coverDate,prism:doi,dc:description,affiliation,author,authkeywords
0,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85045552581,Prevalence of disability in patients with musc...,2017-01-01,10.1097/RHU.0000000000000571,Objective: The aim of this study was to determ...,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",Community | HAQ-DI | Musculoskeletal pain | Ph...
1,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0034771953,Quantum chemical study of effects produced by ...,2001-01-01,NaN,The effects produced in BaTiO3 due to Nb and L...,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",BaTiO 3 | Electron transfer | La electronic st...
2,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85074633643,PRIMER initiative: PRomoting interdisciplinary...,2019-01-01,NaN,The paper describes the PRIMER international i...,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",Information studies | Interdisciplinarity | Kn...
3,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0025824109,Papillary carcinoma in thyroglossal duct remnants,1991-07-01,10.1007/BF00176752,About 100 cases of carcinoma arising in thyrog...,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",Papillary carcinoma | Sistrunk operation | Thy...
4,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0017597578,Mean age of mortality as an indicator of econo...,1977-12-01,NaN,NaN,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",NaN
...,...,...,...,...,...,...,...,...,...,...
242,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0037330579,Predictors and nutritional consequences of int...,2003-02-01,10.1093/tropej/49.1.17,"The study investigated the prevalence, risk fa...","[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",NaN
243,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0037673926,Snakebites in the rainforests of Ecuador,2003-01-01,10.1007/s00268-002-6552-9,Epidemiologic information about snakebites in ...,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",NaN
244,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:30444435957,Stereotactic surgery and radiosurgery in intra...,2005-01-01,NaN,NaN,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",NaN
245,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:0032790944,Epilepsy and neurocysticercosis in an Andean c...,1999-09-02,10.1093/ije/28.4.799,Background: Taenia solium neurocysticercosis (...,"[{'@_fa': 'true', 'affiliation-url': 'https://...","[{'@_fa': 'true', 'author-url': 'https://api.e...",Developing countries | Epilepsy | Neurocystice...


<hr>

<hr>

<h3><strong>Parte 4: Verificar que afiliciones no fueron obtenidas con la API Article Search<strong></h3>

<h4 style="text-align:justify;">Paso 1: Mostrar las afiliaciones extraías</h4>

In [40]:
dfAffiliations.head(2)

,@_fa,prism:url,dc:identifier,affiliation-name,document-count,city,country
0,true,https://api.elsevier.com/content/affiliation/a...,AFFILIATION_ID:60072059,Universidad San Francisco de Quito,3731,Quito,Ecuador
1,true,https://api.elsevier.com/content/affiliation/a...,AFFILIATION_ID:60072054,Escuela Politécnica Nacional,3194,Quito,Ecuador


In [41]:
affiliationsIdsDiff2 = np.setdiff1d( affiliationsIdsSearch, affiliationsIdsRetrieval)
len(affiliationsIdsDiff2)

29

In [42]:
Utils.recastDfAffiliations(dfAffiliations)

In [43]:
for index, row in dfAffiliations.iterrows():
    if row['identifier'] in affiliationsIdsDiff2:
        print('identifier:', row['identifier'])
        print('affiliation_name:', row['affiliation_name'])
        print('city:', row['city'])
        print('country:', row['country'])
        print('document_count:', row['document_count'] ,'\n')

identifier: 60073131
affiliation_name: Spanish Agency for International Cooperation
city: Quito
country: Ecuador
document_count: 4 

identifier: 127084164
affiliation_name: EP Petroecuador
city: nan
country: Ecuador
document_count: 4 

identifier: 128014213
affiliation_name: Yachay City of Knowledge
city: Urcuqui
country: Ecuador
document_count: 1 

identifier: 127847342
affiliation_name: CIDFAE-Ambato
city: Ambato
country: Ecuador
document_count: 1 

identifier: 127703805
affiliation_name: Neuralcom Ecuador
city: nan
country: Ecuador
document_count: 1 

identifier: 127186550
affiliation_name: Hospital Duran i Reynals
city: Duran
country: Ecuador
document_count: 1 

identifier: 127087704
affiliation_name: Pediatric Surgery Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecuador)
city: Guayaquil
country: Ecuador
document_count: 1 

identifier: 127087683
affiliation_name: Pediatric Urology Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecu

<h4 style="text-align:justify;">Paso 2: Razones</h4>

<table>
  <tr>
    <th>ID</th>
    <th>Num Doc</th>
    <th>Razón</th>
    <th>Afiliaci*ón</th>
    <th>Afiliacion en artículos</th>
  </tr>
  <tr>
    <td>60073131</td>
    <td>4-4</td>
    <td>La afiliacion no concuerda. El pais de la afiliación está mal etiquetada. No es una afiliacion ecuatoriana</td>
    <td>Spanish Agency for International Cooperation, Quito, Ecuador</td>
    <td>Agence Espagnole Coop. I., Bata, Equatorial Guinea</td>
  </tr>
  <tr>
    <td>127084164</td>
    <td>4-4</td>
    <td>La afiliacion no concuerda. La afiliación de los artículos no tienen país. Si es una afiliación ecuatoriana</td>
    <td>EP Petroecuador, Ecuador</td>
    <td>EP PetroEcuador</td>
  </tr>
  <tr>
    <td>128014213</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos está mal etiquetada. Si es una afiliación ecuatoriana</td>
    <td>Yachay City of Knowledge, Urcuqui, Ecuador</td>
    <td>School of Chemistry, Yachat Tech, Yachay City of Knowledge, 100119 Urcuqui, Ecuador, http://yachaytech.edu.ec/</td>
  </tr>
  <tr>
    <td>127847342</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>CIDFAE-Ambato, Ambato, Ecuador</td>
    <td>CIDFAE-Ambato</td>
  </tr>
  <tr>
    <td>127703805</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Neuralcom Ecuador, Ecuador</td>
    <td>Neuralcom Ecuador</td>
  </tr>
  <tr>
    <td>127186550</td>
    <td>1-1</td>
    <td>La afiliacion no concuerda. El pais de la afiliación está mal etiquetada. No es una afiliacion ecuatoriana</td>
    <td>Hospital Duran i Reynals, Duran, Ecuador</td>
    <td>Hereditary Cancer Program, Oncobell Program, Instituto de Investigación Biomédica de Bellvitge (IDIBELL), Hospital Duran i Reynals</td>
  </tr>
  <tr>
    <td>127087704</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Pediatric Surgery Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecuador), Guayaquil, Ecuador</td>
    <td>Pediatric Surgery Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecuador)</td>
  </tr>
  <tr>
    <td>127087683</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Pediatric Urology Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecuador), Guayaquil, Ecuador</td>
    <td>Pediatric Urology Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecuador)</td>
  </tr>
  <tr>
    <td>127087663</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Pathology Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecuador), Guayaquil, Ecuador</td>
    <td>Pathology Department. Dr. Roberto Gilbert Elizalde Pediatric Hospital. Guayaquil (Ecuador)</td>
  </tr>
  <tr>
    <td>126715279</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Unidad Educativa Bilingüe Torremar de Ecuador, Ecuador</td>
    <td>Unidad Educativa Bilingüe Torremar de Ecuador</td>
  </tr>
  <tr>
    <td>126546782</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Betania, Quito, Ecuador</td>
    <td>Ecuador Quito, Sector: la Concepción de Alpahuma, Alangasí, Calle A, Betania, Ref. Urbanización Mirador Del Colegio</td>
  </tr>
  <tr>
    <td>126516483</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Las afiliciones no tienen relaciones con autores en los artículos. Si es una afiliacion ecuatoriana</td>
    <td>Instituto Nacional de Patrimonio Cultural del Ecuador, Ecuador</td>
    <td>Instituto Nacional de Patrimonio Cultural del Ecuador</td>
  </tr>
  <tr>
    <td>126442197</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Las afiliciones no tienen relaciones con autores en los artículos. Si es una afiliacion ecuatoriana</td>
    <td>Repsol Ecuador, Ecuador</td>
    <td>Repsol Ecuador</td>
  </tr>
  <tr>
    <td>126376055</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Studium Theologicum Franciscanum «Cardenal Echeverría» - Ecuador, Ecuador</td>
    <td>Studium Theologicum Franciscanum «Cardenal Echeverría» - Ecuador</td>
  </tr>
  <tr>
    <td>125759326</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Unidad de Cuidados Intensivos. Hospital Instituto Ecuatoriano de Seguridad Social. Ambato. República del Ecuador, Ecuador</td>
    <td>Unidad de Cuidados Intensivos. Hospital Instituto Ecuatoriano de Seguridad Social. Ambato. República del Ecuador</td>
  </tr>
  <tr>
    <td>125591444</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Univ. Agraria del Ecuador, Ecuador</td>
    <td>Facultad Med. Vet. y Zootecnia, Univ. Agraria del Ecuador</td>
  </tr>
  <tr>
    <td>125520629</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Delegates to the LADTR from National Societies of Nephrology from Ecuador, Ecuador</td>
    <td>Delegates to the LADTR from National Societies of Nephrology from Ecuador</td>
  </tr>
  <tr>
    <td>125333502</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Departamento de Investigaciones ‘Dr. Carlos J. Finlay y de Barre’, CLEESS- Sede Ecuador, Ecuador</td>
    <td>Centro Latinoamericano de Estudios Epidemiologicos y Salud Social, Departamento de Investigaciones ‘Dr. Carlos J. Finlay y de Barre’, CLEESS- Sede Ecuador</td>
  </tr>
  <tr>
    <td>125092622</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Schlumberger Ecuador, Ecuador</td>
    <td>Schlumberger Ecuador</td>
  </tr>
  <tr>
    <td>123864366</td>
    <td>1-1</td>
    <td>La afiliación si concuerda. La afilición no tienen relaciones con autores en los artículos. Si es una afiliacion ecuatoriana</td>
    <td>Universidad Laica Elroy Alfaro de Manabí, Ecuador</td>
    <td>School of Medicine, Universidad Laica Elroy Alfaro de Manabí, Manta, Ecuador</td>
  </tr>
  <tr>
    <td>120541476</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afilición de los artículos está mal etiquetada. La afilición no tienen relaciones con autores en los artículos. Si es una afiliacion ecuatoriana</td>
    <td>Universidad de las Fuerzas Armadas ESPE. Ecuador, Ecuador</td>
    <td>Ecuador</td>
  </tr>
  <tr>
    <td>119724180</td>
    <td>1-1</td>
    <td>Razón desconocida. Si es una afilición ecuatoriana</td>
    <td>Epidemiology and Clinical Investigation Center, Quito, Ecuador</td>
    <td>Epidemiology and Clinical Investigation Center, Quito, Ecuador</td>
  </tr>
  <tr>
    <td>115199929</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Universidad Andina Simón Bolívar / Sede Ecuador, Quito, nullEC1701, Ecuador</td>
    <td>Universidad Andina Simón Bolívar / Sede Ecuador</td>
  </tr>
  <tr>
    <td>114835083</td>
    <td>1-1</td>
    <td>La afilción si concuerda. La afilición no tienen relaciones con autores en los artículos. Si es una afiliacion ecuatoriana</td>
    <td>Maternidad Isidro Ayora Quito, Ecuador, Ecuador</td>
    <td>Maternidad Isidro Ayora Quito, Ecuador, Ecuador</td>
  </tr>
  <tr>
    <td>112186378</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Centro de Tecnologías de Información, Campus Gustavo Galindo, Via Perimetral Km. 30.5, Apartado Guayaquil 09-01-5863, Ecuador</td>
    <td>entro de Tecnologias de Información, UNACAR</td>
  </tr>
  <tr>
    <td>106768104</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Centro de Planificacin y de Estudios Sociales, Quito, Ecuador</td>
    <td>Centro de Planificacin y de Estudios Sociales, Quito</td>
  </tr>
  <tr>
    <td>106103191</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos está mal etiquetada. Si es una afiliacion ecuatoriana</td>
    <td>Servicio del Parque National Galápagos, Isla Santa Cruz, Galapagos, Santa Cruz, Ecuador</td>
    <td>Servicio del Parque National Galápagos, Isla Santa Cruz, Galápagos, Ecuador</td>
  </tr>
  <tr>
    <td>105743101</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. Si es una afiliacion ecuatoriana</td>
    <td>Medical Services, Merck Sharp and Dohme, Quito, Ecuador</td>
    <td>Medical Services</td>
  </tr>
  <tr>
    <td>105284102</td>
    <td>1-1</td>
    <td>La afiliación no concuerda. La afiliación de los artículos no tienen país. La afilición no tienen relaciones con autores en los artículos. Si es una afiliacion ecuatoriana</td>
    <td>Castillo Entomological Service, Army of Ecuador, Ecuador</td>
    <td>Castillo Entomological Service, Army of Ecuador</td>
  </tr>
</table>

<h4 style="text-align:justify;">Paso 3: Verificar si alguno de los artículos extraídos pertenece a alguna de las afiliaciones no obtenidas con la API article search</h4>

In [44]:
for index, row in dfArticles.iterrows():
    for affil in row['affiliations']:
        if affil['afid'] in affiliationsIdsDiff2:
            print('identifier:', row['identifier'])
            print('title:', row['title'] ,'\n')

No existen artículos extraídos de las afiliaciones no obtenidas con la API article search